# SEND + MORE = MONEY とは

数式が正しくなるように、各アルファベットに 0 から 9 の異なる数字を入れるというパズル(覆面算)の一つです。このアルファベットの組み合わせ (SEND + MORE = MONEY) は、デュードニーというパズル作家兼数学者によって作られたとのことです。

参考: 「SEND＋MORE＝MONEY」この数学パズルが解けますか？ https://gendai.ismedia.jp/articles/-/77197

以下では、この問題を python-iz (izpyc) を使って解いていきます。

# 準備
## iZ-C の dll を準備する
http://www.constraint.org/ja/izc_download/ からダウンロードしたファイルを準備します。(使用するのは「Linux (64bit) 版」です)

アップロードされた tar.gz から dll を抜き出してシステムから参照できるようにします。 

In [1]:
from google.colab import files
# http://www.constraint.org/ja/izc_download/
# http://www.constraint.org/en/izc_download/
files.upload()

!tar zxf izC_v3_7_0_linux64.tar.gz
!cp -a /content/izC_v3_7_0_linux64/lib/libiz* /usr/local/lib
!ldconfig


Saving izC_v3_7_0_linux64.tar.gz to izC_v3_7_0_linux64.tar.gz
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



## python-iz モジュールのインストール
python-iz (モジュール名: izpyc) を Github からインストールします。

In [2]:
!pip install git+https://github.com/tofjw/python-iz.git

  Cloning https://github.com/tofjw/python-iz.git to /tmp/pip-req-build-gvwe6ul6
  Running command git clone -q https://github.com/tofjw/python-iz.git /tmp/pip-req-build-gvwe6ul6
  Created wheel for izpyc: filename=izpyc-0.1.1-cp37-none-any.whl size=7639 sha256=05c7ee46f5d0168ccc2574a36552ecd1e9f4d6c90e82b6c7bec230788c3824be
  Stored in directory: /tmp/pip-ephem-wheel-cache-wfj_3quo/wheels/92/d4/53/eb6ed1f31757baf4e91c0190fb1fa4ea8ff29df7dfd31d28b6
Successfully built izpyc


# 問題を解くためのコード
## 必要なモジュールのインポート

In [3]:
import sys
import izpyc
import izpyc.constraint as C
from izpyc import Int
from izpyc.searcher import SimpleSearcher


## python-iz (izpyc) を使えるようにする
dll を読み込んで初期化を行います。

帰ってくるオブジェクトへの参照をプログラムの終了まで保持しておく必要があるため、変数に代入しておきます。

In [4]:
spc = izpyc.space()

## 領域変数の準備
SEND + MORE = MONEY の各アルファベットに相当する領域変数を準備します。

各変数は 0 から 9 の値をとりえます。

In [5]:
s = Int(0, 9)
e = Int(0, 9)
n = Int(0, 9)
d = Int(0, 9)
m = Int(0, 9)
o = Int(0, 9)
r = Int(0, 9)
y = Int(0, 9)

print(s, e, n, d, m, o, r, y)

{0..9} {0..9} {0..9} {0..9} {0..9} {0..9} {0..9} {0..9}


## 補助変数の準備

SEND を表す変数(v1), MORE を表す変数(v2), MONEY を表す変数(v2)を準備します。

 * v1 = 1000 * s + 100 * e + 10 * n + d
 * v2 = 1000 * m + 100 * o + 10 * r + e
 * v3 = 10000 * m + 1000 * o + 100 * n + 10 * e + y


In [6]:
v1 = C.scal_prod([s, e, n, d], [1000, 100, 10, 1])
v2 = C.scal_prod([m, o, r, e], [1000, 100, 10, 1])
v3 = C.scal_prod([m, o, n, e, y], [10000, 1000, 100, 10, 1])

print(v1, v2, v3)

{0..9999} {0..9999} {0..99999}


## 制約の設定
変数に対して制約を設定します。
 * アルファベットにはすべて異なる数値が入る
 * send, more, money の先頭の数値 (s と m) は 0 ではない
 * money = send + more が成立する

(制約の設定が終わった時点で、変数がとりうる値の範囲が変化します)

In [7]:

C.all_neq([s, e, n, d, m, o, r, y])
s != 0
m != 0
v3 == v1 + v2

print(s, e, n, d, m, o, n, e, y)
print(v1, v2, v3)

9 {2..8} {2..8} {2..8} 1 0 {2..8} {2..8} {2..8}
{9222..9866} {1022..1088} {10244..10888}


## 探索
制約を満たすような変数の値があるかどうかを探索します。

見つかった場合は search() から True が返され、また変数は見つかった値に決定されています。

In [8]:
SimpleSearcher([s, e, d, m, o, r, y]).search()

True

In [9]:
print("  {} {} {} {}".format(s, e, n, d))
print("+ {} {} {} {}".format(m, o, r, e))
print("---------")
print("{} {} {} {} {}".format(m, o, n, e, y))

  9 5 6 7
+ 1 0 8 5
---------
1 0 6 5 2
